In [1]:
#7000 wiki is the set of wikipedia articles crawled
#with the ck12 and spelling city.
#the text is not parsed.



import pandas as pd
import os
path = '/Users/MK/GitHub/the_answer_is/data'
os.chdir(path)
train = pd.read_table('training_set.tsv',sep = '\t')
train.head()

id                                           question correctAnswer  \
0  100001  When athletes begin to exercise, their heart r...             C   
1  100002  Which example describes a learned behavior in ...             C   
2  100003  When two nuclei are combined into one nucleus,...             D   
3  100004  Which is a distinction between an epidemic and...             B   
4  100005  In which way is the orbit of a comet different...             B   

                                             answerA  \
0                                at the tissue level   
1                         smelling the air for odors   
2                                         conversion   
3                        the symptoms of the disease   
4  The orbit of Earth is less circular than the o...   

                                             answerB  \
0                                 at the organ level   
1                             barking when disturbed   
2                                           reaction   
3                     the geographical area affected   
4  The orbit of a comet is more elliptical than t...   

                                             answerC  \
0                                at the system level   
1                                 sitting on command   
2                                            fission   
3                  the species of organisms infected   
4  The orbital period of Earth is much longer tha...   

                                             answerD  
0                              at the cellular level  
1                                    digging in soil  
2                                             fusion  
3            the season in which the disease spreads  
4  The orbital period of a comet is more predicta...

In [3]:
import os
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

#train the model using wikipdeida data                
sentences = MySentences('/Users/MK/GitHub/the_answer_is/data/wikipedia_from_all_ck_words_stemmed') # a memory-friendly iterator
model = gensim.models.Word2Vec(sentences,size=300, window=5, min_count=1, workers=4)
#procedure needed for deleting words not in the training set. 
def only_the_words_in_index( list, index ):
    output = []
    for a, s in enumerate(list):
        if s in index:
            output.append(list[a])
    return output

def string_stemmer(line):
    import re
    from nltk.corpus import stopwords
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer("english")
    line = re.sub(r'[^a-zA-Z ]',r'',line)
    line = line.split()
    line = [word for word in line if word not in stopwords.words('english')]  # remove the stop words. 
    output = []
    for word in line:
        output.append(stemmer.stem(word))     #stem all words 
    output = ' '.join(output)           # join the list to make a string
    return output

In [28]:
import numpy as np 
model = model
myanswer_list = []
myanswer_distance = pd.DataFrame(np.zeros(4).reshape(1,4), columns = ['A','B','C','D'])    #initialize dataframe to store my answers distance


for i in train.index.values:
    length = len(train)
    q = train.ix[i][1]
    a1 = train.ix[i][3]
    a2 = train.ix[i][4]
    a3 = train.ix[i][5]
    a4 = train.ix[i][6]
    
    q = string_stemmer(q)
    a1 = string_stemmer(a1)
    a2 = string_stemmer(a2)
    a3 = string_stemmer(a3)
    a4 = string_stemmer(a4)
    
    q  = q.split()
    a1 = a1.split()
    a2 = a2.split()
    a3 = a3.split()
    a4 = a4.split()
    
    q =  only_the_words_in_index( q, model.index2word)
    a1 = only_the_words_in_index( a1, model.index2word)
    a2 = only_the_words_in_index( a2, model.index2word)
    a3 = only_the_words_in_index( a3, model.index2word)
    a4 = only_the_words_in_index( a4, model.index2word)

    try:
        answer_similarity = np.array([['A',model.n_similarity(q, a1)],['B',model.n_similarity(q, a2)],
                                      ['C',model.n_similarity(q, a3)],['D',model.n_similarity(q, a4)]])
    except:
        print 'Error on ', i, ' and set lengths at random'
        answer_similarity = np.array([['A',np.random.rand()],['B',np.random.rand()],
                                      ['C',np.random.rand()],['D',np.random.rand()]])
        
        myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
        myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
        myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
        myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )
        
        #myanswer_index = answer_similarity[:,1].argsort()[-2]        #get the second best 
        myanswer_index = answer_similarity[:,1].argmax()         #get the maximum similarity 
        myanswer = answer_similarity[myanswer_index][0]
        myanswer_list.append(myanswer)
        continue
    
    myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
    myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
    myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
    myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )
    
    #myanswer_index = answer_similarity[:,1].argsort()[-2]
    myanswer_index = answer_similarity[:,1].argmax()          #get the maximum similarity 
    myanswer = answer_similarity[myanswer_index][0]
    myanswer_list.append(myanswer)
    
    print 'progress: ', i, '/', length 
#for printing out the distance
# myanswer_distance.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_distance.csv', encoding='utf-8')    
train['ck_12_word2vec_answer'] = myanswer_list
    

progress:  0 / 2500
progress:  1 / 2500
progress:  2 / 2500
progress:  3 / 2500
progress:  4 / 2500
progress:  5 / 2500
progress:  6 / 2500
progress:  7 / 2500
progress:  8 / 2500
progress:  9 / 2500
progress:  10 / 2500
progress:  11 / 2500
progress:  12 / 2500
progress:  13 / 2500
progress:  14 / 2500
progress:  15 / 2500
progress:  16 / 2500
progress:  17 / 2500
progress:  18 / 2500
progress:  19 / 2500
progress:  20 / 2500
progress:  21 / 2500
progress:  22 / 2500
progress:  23 / 2500
progress:  24 / 2500
progress:  25 / 2500
progress:  26 / 2500
progress:  27 / 2500
progress:  28 / 2500
progress:  29 / 2500
progress:  30 / 2500
progress:  31 / 2500
progress:  32 / 2500
progress:  33 / 2500
progress:  34 / 2500
progress:  35 / 2500
progress:  36 / 2500
progress:  37 / 2500
progress:  38 / 2500
progress:  39 / 2500
progress:  40 / 2500
progress:  41 / 2500
progress:  42 / 2500
progress:  43 / 2500
progress:  44 / 2500
progress:  45 / 2500
progress:  46 / 2500
progress:  47 / 2500
pr

/Users/MK/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [29]:
train

id                                           question correctAnswer  \
0     100001  When athletes begin to exercise, their heart r...             C   
1     100002  Which example describes a learned behavior in ...             C   
2     100003  When two nuclei are combined into one nucleus,...             D   
3     100004  Which is a distinction between an epidemic and...             B   
4     100005  In which way is the orbit of a comet different...             B   
5     100006  A teacher builds a model of a hydrogen atom. A...             B   
6     100007  Which substance should a student apply to the ...             A   
7     100008  What is the main source of energy for the wate...             A   
8     100009  Which has the greatest effect on aiding the mo...             D   
9     100010  Over time, non-volcanic mountains can form due...             C   
10    100011  The human body has an average, normal temperat...             D   
11    100012  Which is the best explanation of the term ecol...             D   
12    100013  A farmer sprayed his orange trees with a pesti...             A   
13    100014  Researchers work in teams to make cars more fu...             C   
14    100015  Achromatopsia is a genetic disorder in which s...             B   
15    100016  Which of these elements is likely to be found ...             B   
16    100017  Which statement describes a principle scientis...             C   
17    100018  A scientist looks at a graph that shows the pe...             A   
18    100019  Robots can perform tasks that are dangerous fo...             B   
19    100020  A student investigated the percentage of energ...             B   
20    100021  The Voyager II space probe was launched in 197...             C   
21    100022  Which cellular structure allows nutrients to p...             C   
22    100023  Which behavior can help animals establish a te...             C   
23    100024  The brain, spinal cord, and nerves are organs ...             D   
24    100025    Which statement best describes a liquid sample?             A   
25    100026  A condition in bees called "colony collapse di...             B   
26    100027  Which set contains kingdoms that contain only ...             D   
27    100028  When light passes through the lens of your eye...             D   
28    100029  How should researchers share their findings wi...             B   
29    100030  Some ancient Greeks tried to discover the laws...             B   
...      ...                                                ...           ...   
2470  102471  The large, complex molecules found in living o...             B   
2471  102472  The interior of Earth is made up of several ph...             B   
2472  102473             The building of a dam will most likely             B   
2473  102474  On the periodic table, the elements in group 1...             B   
2474  102475  Which of the following statements best explain...             B   
2475  102476  Which is an example of an organism that has be...             D   
2476  102477  A sample in a cylindrical container has a cyli...             B   
2477  102478  At irregular intervals of three to seven years...             C   
2478  102479  Two seismographs are placed 500 km from an act...             D   
2479  102480  A sperm that contains alleles HqT fuses with a...             D   
2480  102481  Fertilization results from the combination of ...             C   
2481  102482  Which invention would a culture living above t...             C   
2482  102483  Nuclear fusion is the source of energy for sta...             A   
2483  102484  Which of the following statements BEST defines...             C   
2484  102485  South America and Africa are two continents se...             D   
2485  102486  Lighting a match and baking a cake are two act...             B   
2486  102487  The most common way humans contribute to the e...             C   
2487  102488  In which taxonomic group ar

In [30]:
train['ck_12_word2vec_correct'] = (train['correctAnswer'] == train['ck_12_word2vec_answer'])

In [31]:
print 'percent correct is ' , train['ck_12_word2vec_correct'].sum(axis =0) / (len(train) + 0.0)

percent correct is  0.3472


Observation1:
1. If you stem both the training documents and the questions,
the percent increased from 0.29 to 0.34.

In [32]:
train.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_7000wiki_stemmed.csv', encoding='utf-8')
myanswer_distance.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_7000wiki_stemmed_distance.csv', encoding='utf-8')  

In [33]:
myanswer_distance

A         B         C         D
0             0.40773  0.449015  0.364753  0.351798
1      0.132015948959  0.167173  0.158588  0.045031
2      0.143791958045  0.320548  0.366202  0.430643
3      0.413309092597  0.150744  0.313763  0.459602
4      0.927427972846  0.931573  0.860712  0.880432
5      0.291560584901  0.303703  0.268605  0.352822
6      0.193136990345  0.200972  0.270078  0.340348
7      0.182266921373  0.250053  0.108819  0.319832
8      0.173724123041  0.121427  0.055383  0.141560
9      0.565458652792  0.557725  0.579249  0.568656
10     0.333217974627  0.303340  0.379710  0.347258
11     0.266221956456  0.203203  0.277337  0.281675
12     0.245803725562  0.302837  0.179860  0.294314
13     0.282694580683  0.522756  0.326887  0.533582
14     0.239651080071  0.268561  0.318599  0.271762
15     0.157590853246  0.292276  0.167029  0.215803
16     0.277025519819  0.338920  0.303232  0.377449
17     0.460597415628  0.312116  0.369773  0.124184
18     0.241567362596  0.227708  0.585794  0.546154
19     0.159362435255  0.141318  0.097277  0.043982
20     0.197630546002  0.333709  0.296239  0.225225
21     0.394856389187  0.244155  0.408358  0.389290
22     0.260154769169  0.301990  0.300414  0.298998
23     0.235828572342  0.143591  0.088346  0.409264
24     0.193543281367  0.168557  0.196655  0.122645
25      0.18196484437  0.368250  0.355615  0.577088
26     0.424212088198  0.414362  0.442550  0.415666
27     0.135743294083  0.119973  0.207027  0.050267
28      0.44351316122  0.579236  0.312669  0.533259
29     0.171490852133  0.271612  0.289984  0.282786
...               ...       ...       ...       ...
2470   0.269799207859  0.340934  0.371673  0.274581
2471   0.294260316414  0.477424  0.507281  0.439993
2472   0.191927241961  0.203260  0.298354  0.167584
2473   0.331341172808  0.281013  0.266833  0.490102
2474   0.767592350252  0.790121  0.762920  0.830150
2475   0.254261016649  0.196628  0.173134  0.101098
2476   0.304430600199  0.303041  0.279931  0.255969
2477   0.382028146972  0.285998  0.321927  0.142993
2478   0.276887282193  0.275355  0.349562  0.358500
2479   0.461426807006  0.439684  0.965339  0.484995
2480   0.648508379015  0.671234  0.607845  0.680849
2481   0.134549016513  0.116992  0.073000  0.117396
2482   0.574506628321  0.207288  0.225671  0.352081
2483   0.227340305516  0.345037  0.334243  0.340487
2484   0.558194317579  0.697794  0.475279  0.541100
2485   0.288852958256  0.285866  0.289997  0.233411
2486  0.0668199310326  0.024944  0.295923  0.254687
2487   0.225339240969  0.268368  0.213493  0.161182
2488   0.414176694088  0.143712  0.286402  0.202487
2489   0.182990197134 -0.002984  0.335834  0.221035
2490   0.369929600645  0.095611  0.442910  0.227731
2491   0.443714055855  0.466682  0.515835  0.391281
2492   0.305144395617  0.348299  0.244773  0.436935
2493   0.195609056659  0.553773  0.427802  0.373387
2494   0.272114578541  0.237837  0.156557  0.399621
2495   0.397715690798  0.403697  0.368068  0.123795
2496   0.165610037935  0.492588  0.020978  0.354172
2497   0.540898638161  0.516977  0.528344 -0.108537
2498   0.270729176619  0.479618  0.285572  0.366496
2499   0.618186538805  0.459169  0.669936  0.427197

[2500 rows x 4 columns]